<a href="https://colab.research.google.com/github/fairyroad/Tensorflow/blob/main/kaggle_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
myfile=files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [2]:
import pandas as pd
import numpy as np

In [13]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
PassengerId=test["PassengerId"]
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [22]:
#object type을 category로 변환시켜서 각각의 문자열들이 포함된 ticket number를 숫자로 인덱싱하기 위한 코드
train["Ticket_type"]=train["Ticket"].apply(lambda x:x[0:3])
train["Ticket_type"]=train["Ticket_type"].astype('category')
train["Ticket_type"]=train["Ticket_type"].cat.codes

test["Ticket_type"]=test["Ticket"].apply(lambda x:x[0:3])
test["Ticket_type"]=test["Ticket_type"].astype('category')
test["Ticket_type"]=test["Ticket_type"].cat.codes

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_type
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,124
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,137
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,148
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,97


In [37]:
full_data=[train,test]

#이름의 길이를 저장시킴
train["Words_Count"]=train["Name"].apply(lambda x:len(x.split()))
test["Words_Count"]=test["Name"].apply(lambda x:len(x.split()))

#cabin에 존재했는지 알아보는 column
train["Has_Cabin"]=train["Cabin"].apply(lambda x: 0 if type(x)==float else 1 )
test["Has_Cabin"]=test["Cabin"].apply(lambda x:0 if type(x)==float else 1)

for dataset in full_data:
  dataset["FamilySize"]=dataset["SibSp"]+dataset["Parch"]+1

for dataset in full_data:
  dataset["IsAlone"]=0
  dataset.loc[dataset["FamilySize"]==1, "IsAlone"]=1

for dataset in full_data:
  dataset["Embarked"]=dataset["Embarked"].fillna("S")

#원래 보통 dataset중에 nan값은 중간값으로 채워줄때가 많음
for dataset in full_data:
  dataset["Fare"]=dataset["Fare"].fillna(train["Fare"].median())

#qcut()은 같은 개수를 가진 구간으로 나눠줌!!
train["CategoricalFare"]=pd.qcut(train["Fare"],4)

#radint는 random숫자 뽑아내기(균일 분포의 정수난수 1개 생성)
for dataset in full_data:
  age_avg=dataset["Age"].mean()
  age_std=dataset["Age"].std()
  age_null_count=dataset["Age"].isnull().sum()
  age_null_random_list=np.random.randint(age_avg-age_std,age_avg+age_std,size=age_null_count)
  dataset["Age"][np.isnan(dataset["Age"])]=age_null_random_list
  dataset["Age"]=dataset["Age"].astype(int)

#cut() 같은 크기의 구간 5개로 분류하기
train["CategoricalAge"]=pd.cut(train["Age"],5)



train.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_type,Words_Count,Has_Cabin,FamilySize,IsAlone,CategoricalFare,CategoricalAge
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,124,4,0,2,0,"(-0.001, 7.91]","(16.0, 32.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,137,7,1,2,0,"(31.0, 512.329]","(32.0, 48.0]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,148,3,0,1,1,"(7.91, 14.454]","(16.0, 32.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,3,7,1,2,0,"(31.0, 512.329]","(32.0, 48.0]"
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,97,4,0,1,1,"(7.91, 14.454]","(32.0, 48.0]"
